In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import itertools
import re
import matplotlib.pyplot as plt
import collections

In [ ]:
import base64
import requests

load the data

In [ ]:
header_url = 'https://raw.githubusercontent.com/Susiesyyyy/Capstone_Disney_AP_Grouping/main/AP%20Visitation%20header_by%20day.txt'
visitation_url = 'https://raw.githubusercontent.com/Susiesyyyy/Capstone_Disney_AP_Grouping/main/AP%20Visitation_20201101_20211031_by%20day.csv'

In [ ]:
req = requests.get(header_url)
req = req.text

In [ ]:
header = req.split("\r")
header = header[0].split("\t")

In [ ]:
visit_df = pd.read_csv(visitation_url, sep=',', header=None)
visit_df.columns = header
visit_df.head()

,source_id,Passtype,Age,MerchSpendLevel,is_renewal,use_time
0,5361,Gold,Adult,<1K,0,2020-11-06 12:01:37.917
1,6604,Gold,Adult,>1K,0,2020-12-11 17:05:15.397
2,6604,Gold,Adult,>1K,0,2021-01-03 10:44:05.687
3,6604,Gold,Adult,>1K,0,2021-01-13 11:47:10.623
4,6604,Gold,Adult,>1K,0,2021-01-28 10:22:03.467


In [ ]:
visit_df.shape

(92564, 6)

In [ ]:
visit_time = visit_df[['source_id','use_time']]

In [ ]:
# visit_time['year1'] = visit_time['year'].astype(str)
# visit_time['month1'] = visit_time['month'].astype(str)
# visit_time['time'] = visit_time['year1'] + visit_time['month1']
# visit_time = visit_time.drop(['year','year1','month','month1'],axis=1)

Make the visit time to visit hour

In [ ]:
visit_time['use_hour'] = visit_time['use_time'].apply(lambda x: x[:13])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
visit_time

,source_id,use_time,use_hour
0,5361,2020-11-06 12:01:37.917,2020-11-06 12
1,6604,2020-12-11 17:05:15.397,2020-12-11 17
2,6604,2021-01-03 10:44:05.687,2021-01-03 10
3,6604,2021-01-13 11:47:10.623,2021-01-13 11
4,6604,2021-01-28 10:22:03.467,2021-01-28 10
...,...,...,...
92559,354954,2020-11-01 11:50:25.353,2020-11-01 11
92560,354954,2021-02-21 14:15:52.797,2021-02-21 14
92561,354954,2021-06-20 14:54:30.807,2021-06-20 14
92562,354954,2021-08-27 15:11:45.997,2021-08-27 15


Load the testing data (WeChat grouping result)

In [ ]:
wechat_group = 'https://raw.githubusercontent.com/Susiesyyyy/Capstone_Disney_AP_Grouping/main/wechat_group.csv'
testing_data = pd.read_csv(wechat_group,header=None)

In [ ]:
testing_data.head()

,0,1,2,3
0,259183,53500,NaN,NaN
1,251084,251079,NaN,NaN
2,284757,277972,NaN,NaN
3,272049,272048,NaN,NaN
4,299998,299999,NaN,NaN


In [ ]:
testing_data.shape

(568, 4)

In [ ]:
visitation_id = list(np.unique(visit_time.source_id))
testing_id = list(np.unique(testing_data))

Get the overlapped source_id

In [ ]:
overlap_id = [x for x in visitation_id + testing_id if x in visitation_id and x in testing_id]
overlap_array = np.unique(np.array(overlap_id))
overlap_list = list(overlap_array)

In [ ]:
# overlap_list

In [ ]:
len(overlap_list)

1167

In [ ]:
overlap_df = visit_time.loc[visit_time['source_id'].isin(overlap_list),]

In [ ]:
overlap_df

,source_id,use_time,use_hour
96,18919,2020-11-01 13:50:38.513,2020-11-01 13
97,18919,2021-01-28 14:47:41.943,2021-01-28 14
98,18919,2021-02-07 15:41:50.020,2021-02-07 15
99,18919,2021-08-30 13:38:26.233,2021-08-30 13
100,18919,2021-09-03 17:46:51.713,2021-09-03 17
...,...,...,...
92559,354954,2020-11-01 11:50:25.353,2020-11-01 11
92560,354954,2021-02-21 14:15:52.797,2021-02-21 14
92561,354954,2021-06-20 14:54:30.807,2021-06-20 14
92562,354954,2021-08-27 15:11:45.997,2021-08-27 15


Group the source_id by visit hour

In [ ]:
same_hour_id = overlap_df.groupby('use_hour')['source_id'].apply(lambda x: list(np.unique(x)))

In [ ]:
same_hour_id = same_hour_id.to_frame().reset_index()

In [ ]:
same_hour_id

,use_hour,source_id
0,2020-11-01 08,"[248016, 248017]"
1,2020-11-01 09,"[250398, 257422, 257435, 258100, 259069, 354533]"
2,2020-11-01 10,"[250396, 250530, 250534, 255246, 255247, 25722..."
3,2020-11-01 11,"[44794, 44800, 257288, 257289, 258228, 354627,..."
4,2020-11-01 12,"[43803, 43804, 45901, 45902, 47365, 47366, 247..."
...,...,...
2214,2021-10-30 18,"[286167, 286168]"
2215,2021-10-31 07,"[59425, 278560, 278562, 290935]"
2216,2021-10-31 08,"[262990, 262993, 290936]"
2217,2021-10-31 09,"[57535, 57536]"


Network Model

In [ ]:
from itertools import combinations
G = nx.Graph()

for i in range(len(same_hour_id)):
    for j in range(len(same_hour_id['source_id'][i])):
        if str(same_hour_id['source_id'][i][j]) not in G.nodes():
            G.add_node(str(same_hour_id['source_id'][i][j]),weight = 1)
        else:
            G.nodes[str(same_hour_id['source_id'][i][j])]['weight'] += 1
            
    edge_combinations = list(combinations(same_hour_id['source_id'][i],2))
    for combination in edge_combinations:
        if (str(combination[0]),str(combination[1])) not in G.edges():
            G.add_edge(str(combination[0]),str(combination[1]),weight=1)
        else:
            G.edges[str(combination[0]),str(combination[1])]['weight'] += 1
            
sorted_same_month_group = sorted(G.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)

In [ ]:
groups = []

for node1 in G.nodes():
    node1_weight = G.nodes[node1]['weight']
    
    if len(G.edges(node1,data=True)) > 0:
    
        max_value = sorted(G.edges(node1,data=True),key=lambda x: x[2]['weight'],reverse=True)[0][-1]
        max_value = [int(value) for key, value in max_value.items()][0]
    
        if max_value > 1:
            node2 = sorted(G.edges(node1,data=True),key= lambda x: x[2]['weight'],reverse=True)[0][1]
            node2_weight = G.nodes[node2]['weight']
    
            group_weight = sorted(G.edges(node1,data=True),key= lambda x: x[2]['weight'],reverse=True)[0][-1]
            group_weight = [int(value) for key, value in group_weight.items()][0]
    
            node1_neighbor = []
            node2_neighbor = []

            for nbr in G.neighbors(node1):
                threshold_weight = 90/100*node1_weight
                if G.edges[node1, nbr]['weight'] > threshold_weight:
                    if nbr != node2:
                        node1_neighbor.append(nbr)

            for nbr in G.neighbors(node2):
                threshold_weight = 90/100*node2_weight
                if G.edges[node2, nbr]['weight'] > threshold_weight: 
                    if nbr != node1:
                        node2_neighbor.append(nbr)

            group_neighbor = node1_neighbor+node2_neighbor
            group_neighbor = list(set(group_neighbor))            

            current_group = []
            current_group.append(node1)
            current_group.append(node2)
            current_group += group_neighbor

            if current_group not in groups:
                groups.append(current_group)
    

In [ ]:
unique_groups = [list(x) for x in set(frozenset(i) for i in [set(i) for i in groups])]
wechat_group = [testing_data.loc[i] for i in range(testing_data.shape[0])]

Calculate the accuracy

In [ ]:
for group in wechat_group:
  group.dropna(inplace = True)
wechat_group_list = [list(n) for n in wechat_group]

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:5258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(result)


In [ ]:
unique_groups_int = []
for group in unique_groups:
  group = [int(x) for x in group]
  unique_groups_int.append(group)

In [ ]:
group_set = set([frozenset(x) for x in unique_groups_int])
wechat_set = set([frozenset(x) for x in wechat_group])
difference_set = group_set.difference(wechat_set)
difference_set2 = wechat_set.difference(group_set)
print(len(group_set), len(wechat_set), len(difference_set), len(difference_set2))
print(1 - len(difference_set2)/len(wechat_set))

580 568 44 32
0.9436619718309859


In [ ]:
#group_set

In [ ]:
#wechat_set

In [ ]:
#difference_set

In [ ]:
accuracy = {}
for group in unique_groups:
  group = [int(x) for x in group]
  similarity_count = []
  set1 = set(group)
  for group2 in wechat_group_list:
    set2 = set(group2)
    difference_set = set2.difference(set1)
    similarity_count.append(1 - (len(difference_set)/len(set2)))
  accuracy[str(group)] = [max(similarity_count), str(list(wechat_group[similarity_count.index(max(similarity_count))]))]

In [ ]:
partial_correct = []
for value in list(accuracy.values()):
  if value[0] < 1:
    partial_correct.append(value)

In [ ]:
sum = 0
for value in list(accuracy.values()):
  sum += value[0]
avg_accuracy = sum/len(accuracy)
print(avg_accuracy)

0.979454022988506


In [ ]:
accuracy

{'[186557, 186559]': [1.0, '[186557.0, 186559.0]'],
 '[187027, 60325]': [1.0, '[187027.0, 60325.0]'],
 '[187361, 187363, 187362]': [1.0, '[187363.0, 187361.0, 187362.0]'],
 '[187424, 63118]': [1.0, '[63118.0, 187424.0]'],
 '[187686, 187687]': [1.0, '[187687.0, 187686.0]'],
 '[188750, 188751]': [1.0, '[188750.0, 188751.0]'],
 '[18919, 18920]': [1.0, '[18920.0, 18919.0]'],
 '[247703, 42084]': [0.6666666666666667, '[42085.0, 42084.0, 247703.0]'],
 '[248017, 248016]': [0.6666666666666667, '[248018.0, 248016.0, 248017.0]'],
 '[248018, 248017, 248016]': [1.0, '[248018.0, 248016.0, 248017.0]'],
 '[248281, 248279]': [1.0, '[248279.0, 248281.0]'],
 '[249634, 249635]': [1.0, '[249634.0, 249635.0]'],
 '[249678, 249676]': [1.0, '[249676.0, 249678.0]'],
 '[249766, 251119]': [1.0, '[249766.0, 251119.0]'],
 '[249806, 249807]': [1.0, '[249806.0, 249807.0]'],
 '[250206, 250207]': [1.0, '[250206.0, 250207.0]'],
 '[250396, 250398]': [1.0, '[250396.0, 250398.0]'],
 '[250479, 250476]': [1.0, '[250479.0, 25

In [ ]:
for group in unique_groups:
  if '278983' in group:
    print(group)

['278983', '278985']


In [ ]:
for group in wechat_group_list:
  if 274410 in group:
    print(group)

[274410.0, 274409.0]


In [ ]:
wechat_group_list

In [ ]:
overlap_id

In [ ]:
print(len(groups),len(unique_groups))

1139 581


In [ ]:
len_unique_groups = []

for i in unique_groups:
  a = len(i)
  len_unique_groups.append(a)

In [ ]:
counter = collections.Counter(len_unique_groups)
print(counter)

Counter({2: 547, 3: 29, 4: 4})


Try on the total visitation data

In [ ]:
all_hour_id = visit_time.groupby('use_hour')['source_id'].apply(lambda x: list(np.unique(x)))

In [ ]:
all_hour_id = all_hour_id.to_frame().reset_index()

In [ ]:
all_hour_id

,use_hour,source_id
0,2020-11-01 07,"[42865, 47048, 249681, 249902]"
1,2020-11-01 08,"[45638, 247710, 248016, 248017, 249792, 250315..."
2,2020-11-01 09,"[38482, 40240, 247740, 249052, 249188, 249367,..."
3,2020-11-01 10,"[17118, 40148, 40867, 45831, 46867, 248244, 24..."
4,2020-11-01 11,"[27333, 29734, 37609, 41403, 42864, 44696, 447..."
...,...,...
4604,2021-10-31 12,"[38829, 44833, 47690, 53808, 57413, 60292, 707..."
4605,2021-10-31 13,"[42479, 51118, 67069, 67742, 72132, 258136, 27..."
4606,2021-10-31 14,"[44566, 64398, 255526, 260587, 273950, 274066,..."
4607,2021-10-31 15,"[38642, 62020, 251810, 272685, 283300, 288878]"


In [ ]:
explode_id = all_hour_id.explode('source_id', ignore_index = True)

In [ ]:
network_id = explode_id.source_id

In [ ]:
wechat_df = pd.DataFrame(wechat_group)
wechat_df = wechat_df.melt()

In [ ]:
wechat_df.dropna(inplace = True)

In [ ]:
wechat_id = wechat_df.value

In [ ]:
wechat_id_set = set(wechat_id)
network_id_set = set(network_id)

In [ ]:
for id in network_id_set:
  for group in wechat_group:
    if id in group:
      print(id, group)
    

In [ ]:
network_id_set.intersection(wechat_id_set)

{299010.0,
 299011.0,
 63492.0,
 63494.0,
 51207.0,
 262151.0,
 51208.0,
 69641.0,
 270347.0,
 69643.0,
 268301.0,
 67597.0,
 270348.0,
 268302.0,
 278547.0,
 278548.0,
 272408.0,
 270360.0,
 278553.0,
 262171.0,
 272409.0,
 262170.0,
 278552.0,
 247839.0,
 278560.0,
 59425.0,
 270368.0,
 280611.0,
 65571.0,
 280607.0,
 278562.0,
 71719.0,
 256041.0,
 71721.0,
 282668.0,
 282669.0,
 266285.0,
 290865.0,
 290866.0,
 276531.0,
 272436.0,
 274485.0,
 276532.0,
 274484.0,
 258100.0,
 290868.0,
 256049.0,
 254011.0,
 272439.0,
 254013.0,
 276541.0,
 290879.0,
 290880.0,
 276542.0,
 280643.0,
 41032.0,
 41033.0,
 270412.0,
 284757.0,
 53337.0,
 270426.0,
 53338.0,
 270428.0,
 290909.0,
 290908.0,
 45158.0,
 45159.0,
 274540.0,
 67700.0,
 268405.0,
 268406.0,
 67703.0,
 290935.0,
 282744.0,
 282743.0,
 290936.0,
 276604.0,
 276605.0,
 276606.0,
 45187.0,
 45189.0,
 282758.0,
 282759.0,
 276618.0,
 276620.0,
 276621.0,
 262290.0,
 286875.0,
 262304.0,
 262306.0,
 270499.0,
 270500.0,
 65702.0,

In [ ]:
from itertools import combinations
G1 = nx.Graph()

for i in range(len(all_hour_id)):
    for j in range(len(all_hour_id['source_id'][i])):
        if str(all_hour_id['source_id'][i][j]) not in G1.nodes():
            G1.add_node(str(all_hour_id['source_id'][i][j]),weight = 1)
        else:
            G1.nodes[str(all_hour_id['source_id'][i][j])]['weight'] += 1
            
    edge_combinations = list(combinations(all_hour_id['source_id'][i],2))
    for combination in edge_combinations:
        if (str(combination[0]),str(combination[1])) not in G1.edges():
            G1.add_edge(str(combination[0]),str(combination[1]),weight=1)
        else:
            G1.edges[str(combination[0]),str(combination[1])]['weight'] += 1
            
sorted_same_hour_group_all = sorted(G1.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)

In [ ]:
groups_all = []

for node1 in G1.nodes():
    node1_weight = G1.nodes[node1]['weight']
    
    if len(G1.edges(node1,data=True)) > 0:
    
        max_value = sorted(G1.edges(node1,data=True),key=lambda x: x[2]['weight'],reverse=True)[0][-1]
        max_value = [int(value) for key, value in max_value.items()][0]
    
        if max_value > 1:
            node2 = sorted(G1.edges(node1,data=True),key= lambda x: x[2]['weight'],reverse=True)[0][1]
            node2_weight = G1.nodes[node2]['weight']
    
            group_weight = sorted(G1.edges(node1,data=True),key= lambda x: x[2]['weight'],reverse=True)[0][-1]
            group_weight = [int(value) for key, value in group_weight.items()][0]
    
            node1_neighbor = []
            node2_neighbor = []

            for nbr in G1.neighbors(node1):
                threshold_weight = 90/100* (node1_weight + node2_weight)
                if G1.edges[node1, nbr]['weight'] > threshold_weight:
                    if nbr != node2:
                        node1_neighbor.append(nbr)

            for nbr in G1.neighbors(node2):
                threshold_weight = 90/100* (node1_weight + node2_weight)
                if G1.edges[node2, nbr]['weight'] > threshold_weight: 
                    if nbr != node1:
                        node2_neighbor.append(nbr)

            group_neighbor = node1_neighbor+node2_neighbor
            group_neighbor = list(set(group_neighbor))            

            current_group = []
            current_group.append(node1)
            current_group.append(node2)
            current_group += group_neighbor

            if current_group not in groups_all:
                groups_all.append(current_group)

In [ ]:
unique_groups_all = [list(x) for x in set(frozenset(i) for i in [set(i) for i in groups_all])]

In [ ]:
len(unique_groups_all)

7507

In [ ]:
len_unique_groups = []

for i in unique_groups_all:
  a = len(i)
  len_unique_groups.append(a)

In [ ]:
counter = collections.Counter(len_unique_groups)

In [ ]:
print(counter)

Counter({2: 6763, 3: 609, 4: 42, 5: 14, 6: 9, 7: 8, 10: 6, 9: 5, 8: 5, 16: 2, 11: 2, 14: 2, 23: 1, 13: 1, 12: 1, 24: 1})


In [ ]:
for id in wechat_id_set:
  for group in unique_groups_all:
    if id in group:
      print(id, group)

In [ ]:
for group in wechat_group:
  group.dropna(inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:5258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(result)


In [ ]:
wechat_group_list = [list(n) for n in wechat_group]

In [ ]:
accuracy = {}
for group in unique_groups_all:
  similarity_count = []
  set1 = set(group)
  for group2 in wechat_group_list:
    set2 = set(group2)
    difference_set = set2.difference(set1)
    similarity_count.append(1 - len(difference_set)/len(set2))
  accuracy[str(group)] = [max(similarity_count), str(list(wechat_group[similarity_count.index(max(similarity_count))]))]

In [ ]:
for group in unique_groups_all:
  if '276680' in group:
    print(group)

['276681', '276679', '276680', '277356']
['276681', '276679', '276680']


In [ ]:
G1.edges['280491', '290727']['weight']

2

In [ ]:
G1.nodes['265619']['weight']

34

In [ ]:
for group in unique_groups_all:
  if '47740' in group:
    print(group)

['47740', '275726']
['47740', '58299']
['47740', '267652']
['47740', '249127']
['47740', '269125']
['47740', '287401']
['47740', '255209']
['47740', '279666']
['47740', '280491']
['47740', '290367']
['47740', '265619']
['47740', '268312']
['47740', '71626']
['47740', '283646']
['47740', '284750']
['47740', '43357']
['47740', '270020']
['47740', '274603']
['267631', '47740']
['47740', '274183']
['47740', '270135']
['47740', '269066']
['273719', '47740']
['47740', '287585']
['47740', '282092']
['263169', '47740']
['47740', '265650']
['47740', '56082']
['47740', '56939']
['47740', '268241']
['47740', '51785']


In [ ]:
G1.edges['248016', '248018']['weight']

9

In [ ]:
G1.nodes['248016']['weight']

12

In [ ]:
accuracy

In [ ]:
max(list(accuracy.values()))

[0.0, '[259183.0, 53500.0]']

In [ ]:
difference_set3 = wechat_set.difference(set(map(tuple, unique_groups_all)))

In [ ]:
len(difference_set3)

568

In [ ]:
len(wechat_set)

In [ ]:
len(wechat_group)

568